In [23]:
import pandas
import googleapiclient
from googleapiclient.discovery import build
from secrets import API_KEY
youtube = build('youtube', 'v3', developerKey=API_KEY)
analytics = build('youtubeAnalytics', 'v1', developerKey=API_KEY)

pyconau_channels = youtube.channels().list(forUsername='PyConAU', part='contentDetails').execute()
uploads_list_id = pyconau_channels['items'][0]['contentDetails']['relatedPlaylists']['uploads']

# Retrieve the list of videos uploaded to the PyConAU channel.
playlistitems_list_request = youtube.playlistItems().list(
    playlistId=uploads_list_id,
    part="snippet",
    #maxResults=10
  )

data1 = []
columns1 = ('id', 'title', 'published')
while playlistitems_list_request:

    playlistitems_list_response = playlistitems_list_request.execute()

    for playlist_item in playlistitems_list_response["items"]:
        title = playlist_item["snippet"]["title"]
        published = playlist_item["snippet"]["publishedAt"]
        video_id = playlist_item["snippet"]["resourceId"]["videoId"]
        data1.append((video_id, title, published))

        playlistitems_list_request = youtube.playlistItems().list_next(
          playlistitems_list_request, playlistitems_list_response)

df1 = pandas.DataFrame(data1, columns=columns1)

In [28]:
df1.head()

,id,title,published
0,Wnj31Nejstc,Are they ready? A quick review of graph databa...,2015-08-29T07:23:39.000Z
1,c16jwvOmKxM,Cooking with Cryptography by Fraser Tweedale,2015-08-17T22:18:57.000Z
2,2e9dMr5sErs,Education Miniconf Closing Keynote by Carrie A...,2015-08-17T04:49:16.000Z
3,9_LnOTFDNUk,¡Mangle You A Python Interpreter for New Behav...,2015-08-16T13:09:58.000Z
4,0FOQB4PZVe0,Privsep: A New Security Isolation Mechanism fo...,2015-08-16T11:05:40.000Z


In [18]:
data2 = []
columns2 = ('id', 'viewCount', 'likeCount', 'faveCount')

start = 0
while start < len(df1):
    ids = ','.join(df1['id'][start:start + 50])
    all_videos = youtube.videos().list(id=ids, part='id,statistics').execute()
    for video in all_videos['items']:
        stats = video['statistics']
        data2.append((video['id'], stats['viewCount']), stats['likeCount'], stats['favoriteCount']))
    start += 50

df2 = pandas.DataFrame(data2, columns=columns2)

In [22]:
df2.head()

,id,viewCount,likeCount,faveCount
0,Wnj31Nejstc,366,2,0
1,c16jwvOmKxM,194,0,0
2,2e9dMr5sErs,100,1,0
3,9_LnOTFDNUk,90,1,0
4,0FOQB4PZVe0,68,1,0


In [58]:
videos = pandas.merge(df1, df2, on='id')
videos['published'] = pandas.to_datetime(videos['published'])
videos[['viewCount', 'likeCount', 'faveCount']] = videos[['viewCount', 'likeCount', 'faveCount']].astype(int)
videos.head()

,id,title,published,viewCount,likeCount,faveCount
0,Wnj31Nejstc,Are they ready? A quick review of graph databa...,2015-08-29 07:23:39,366,2,0
1,c16jwvOmKxM,Cooking with Cryptography by Fraser Tweedale,2015-08-17 22:18:57,194,0,0
2,2e9dMr5sErs,Education Miniconf Closing Keynote by Carrie A...,2015-08-17 04:49:16,100,1,0
3,9_LnOTFDNUk,¡Mangle You A Python Interpreter for New Behav...,2015-08-16 13:09:58,90,1,0
4,0FOQB4PZVe0,Privsep: A New Security Isolation Mechanism fo...,2015-08-16 11:05:40,68,1,0


In [59]:
videos.sort_values(by=['likeCount', 'viewCount'], ascending=False).head()

,id,title,published,viewCount,likeCount,faveCount
268,AYjPIMe0BhA,Web micro-framework battle - Richard Jones,2011-08-22 14:29:11,17070,161,0
214,hqOsfS3dP9w,Modern scientific computing and big data analy...,2013-07-10 14:55:25,48088,144,0
212,RQ-lIU0jWwg,Making games in pygame,2013-07-11 00:45:36,25368,103,0
194,yI7NYgP54sw,Tinkering with Tkinter,2013-07-11 00:45:36,15767,101,0
225,H2yfXnUb1S4,Don't Do This,2013-07-10 09:18:16,16630,67,0


In [60]:
videos.sort_values(by=['faveCount'], ascending=False).head()

,id,title,published,viewCount,likeCount,faveCount
0,Wnj31Nejstc,Are they ready? A quick review of graph databa...,2015-08-29 07:23:39,366,2,0
218,UwuMclgqtAQ,Growing Open Source Seeds,2013-07-10 13:22:36,255,1,0
198,-AGgugxn4RM,Software Carpentry arrives Down Under!,2013-07-11 00:45:36,230,3,0
197,uI39nXHLVj8,The life of a web request - techniques for mea...,2013-07-11 00:45:36,513,4,0
196,wWlmpoSN34I,Transitioning from PHP to Django on the sly,2013-07-11 00:45:36,977,15,0


In [63]:
videos[videos['faveCount'] > 0]

,id,title,published,viewCount,likeCount,faveCount
